In [1]:
import sys
sys.path.append("D:\\work\\searchformer\\searchformer")

In [2]:
from searchformer.sokoban import SokobanTrace,SokobanRenderer,SimpleSokobanTokenizer,sokoban_state_to_pretty_string
import pygame
import json

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
with open("D:\\work\\searchformer\\7x7_sokoban_trace.json") as file:
    trace=json.load(file)

In [5]:
start_state=trace['sokoban_start']

In [6]:
actions=[]
for action in trace['trace']:
    # if action['action']=='create' or action['action']=='close':
    if action['action']=='plan':
        state=action['state']['state']
        actions.append(state)

In [7]:
len(actions)

15

In [8]:
renderer=SokobanRenderer(width=6,height=6,record_dir="D:\\work\\searchformer\\record_dir",assets_dir="D:\\work\\searchformer\\sokoban")
pygame.init()
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    for action in actions:
        renderer.render(game_state=action)
        pygame.display.flip()
        clock.tick(1)  # Optional: limit to 30 FPS
    # renderer.img_to_file()
    

pygame.quit()

: 